In [ ]:
import prices_by_scrap
import pandas as pd
import numpy as np
from datetime import date 

CSV_IN = "result/prices.csv"
CSV_OUT = "result/average_by_month.csv"
ARGUMENTS = "Frankreich -l WARNING -o prices.csv -f result"
COLUMNS = ["Date","Country","Name","Price in euros per MW"]
DATE_FORMAT = "%Y-%m-%d"


In [ ]:
# Execute program
prices_by_scrap.main(ARGUMENTS)

In [ ]:
# Process data
df = pd.read_csv(CSV_IN, sep=';', parse_dates=False)

df[COLUMNS[3]] = pd.to_numeric(df[COLUMNS[3]].str.replace(',', '.'))/4
df[COLUMNS[0]] = pd.to_datetime(df[COLUMNS[0]], format=DATE_FORMAT)
df['month'] = pd.DatetimeIndex(df[COLUMNS[0]]).month
df['year'] = pd.DatetimeIndex(df[COLUMNS[0]]).year
table = pd.pivot_table(df, values=COLUMNS[3], index=[COLUMNS[0]], aggfunc=["min", "max", "mean"], margins=False)
table.reset_index(inplace = True)
table[COLUMNS[0]] = pd.to_datetime(table[COLUMNS[0]], format="%d/%m/%Y")
table['month'] = pd.DatetimeIndex(table[COLUMNS[0]]).month
table['year'] = pd.DatetimeIndex(table[COLUMNS[0]]).year
table.set_index(COLUMNS[0], inplace=True)
table2 = pd.pivot_table(table, values=['min', 'max', 'mean'], index=['year', 'month'], aggfunc="mean")
table2 = table2[['min', 'mean', 'max']]
table2 = table2.round(decimals=1)
print("Moyennes mensuelles des min/max journaliers et des moyennes journalières\n")
print(table2)
print("\n\n")

table2.to_csv(CSV_OUT, sep =';')

todays_date = date.today()
table = pd.pivot_table(df[ (df['year'] == todays_date.year) & (df['month'] == todays_date.month) ],
                       values=COLUMNS[3], index=[COLUMNS[0]], aggfunc=["min", "max", "mean"], margins=True)
table = table[['min', 'mean', 'max']]
table = table.round(decimals=1)
print("Zoom mois en cours\n")
print(table)